## Model buildiing
- Notebook used to validate feature selection for logistic regression 
- Find features
- Build model
- Check accuracy score
- Output pickle model 
- output empty datrame with dummy columns

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas as pd
import pickle
import seaborn as sns
import numpy as np

#### Import dataset

In [2]:
df = pd.read_csv('../backend/mapping_heat/fixtures/pitching_data.csv')

In [78]:
df.columns

Index(['index', 'pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
     

In [4]:
pd.set_option('display.max_columns', None)
df.head()

,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,613,FF,9/27/20,91.6,2.31,6.19,Brad Hand,605137,543272,field_out,hit_into_play,NaN,NaN,NaN,NaN,1.0,Josh Bell flies out to right fielder Jordan Lu...,R,R,L,CLE,PIT,X,9.0,fly_ball,1,0,2020,0.9,1.3,-0.30,3.31,NaN,NaN,NaN,2,9,Top,217.94,101.96,NaN,NaN,547379,NaN,NaN,-8.326379,-133.205861,-4.261343,11.890496,25.776284,-15.718337,3.73,1.77,315.0,88.4,32.0,90.8,2524.0,5.6,631538,543272,547379,467793,514917,608070,596019,640458,592261,656669,54.90,0.043,0.049,0.0,1.0,0.0,0.0,3.0,77,2,4-Seam Fastball,8,6,6,8,6,8,6,8,Standard,Standard
1,635,SL,9/27/20,80.4,2.45,6.12,Brad Hand,605137,543272,NaN,ball,NaN,NaN,NaN,NaN,13.0,NaN,R,R,L,CLE,PIT,B,NaN,NaN,0,0,2020,-1.1,0.0,-0.86,1.31,NaN,NaN,NaN,2,9,Top,NaN,NaN,NaN,NaN,547379,NaN,NaN,-5.260466,-116.999311,-3.880876,-8.699110,21.709584,-31.961590,3.86,1.91,NaN,NaN,NaN,79.2,2527.0,5.4,631538,543272,547379,467793,514917,608070,596019,640458,592261,656669,55.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77,1,Slider,8,6,6,8,6,8,6,8,Standard,Standard
2,637,SL,9/27/20,80.8,2.27,6.09,Brad Hand,592567,543272,strikeout,swinging_strike,NaN,NaN,NaN,NaN,13.0,Colin Moran strikes out swinging.,R,L,L,CLE,PIT,S,2.0,NaN,0,2,2020,-1.4,-0.1,-1.90,1.26,NaN,NaN,NaN,1,9,Top,NaN,NaN,NaN,NaN,547379,NaN,NaN,-6.545887,-117.560799,-3.823399,-11.851736,21.561248,-33.038975,3.59,1.70,NaN,NaN,NaN,79.9,2357.0,5.6,631538,543272,547379,467793,514917,608070,596019,640458,592261,656669,54.93,NaN,NaN,0.0,1.0,0.0,0.0,NaN,76,4,Slider,8,6,6,8,6,8,6,8,Infield shift,Standard
3,660,SL,9/27/20,80.5,2.41,6.12,Brad Hand,592567,543272,NaN,foul,NaN,NaN,NaN,NaN,13.0,NaN,R,L,L,CLE,PIT,S,NaN,NaN,0,2,2020,-1.7,0.0,-0.90,1.80,NaN,NaN,NaN,1,9,Top,NaN,NaN,NaN,NaN,547379,NaN,NaN,-4.073822,-117.239671,-2.937641,-14.751489,23.216014,-31.542665,3.59,1.70,NaN,NaN,NaN,79.1,2405.0,5.3,631538,543272,547379,467793,514917,608070,596019,640458,592261,656669,55.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76,3,Slider,8,6,6,8,6,8,6,8,Infield shift,Standard
4,661,SL,9/27/20,81.4,2.35,6.14,Brad Hand,592567,543272,NaN,swinging_strike,NaN,NaN,NaN,NaN,13.0,NaN,R,L,L,CLE,PIT,S,NaN,NaN,0,1,2020,-1.7,0.0,-0.98,2.25,NaN,NaN,NaN,1,9,Top,NaN,NaN,NaN,NaN,547379,NaN,NaN,-4.216575,-118.535362,-2.219346,-14.860351,22.417236,-31.546104,3.59,1.70,NaN,NaN,NaN,80.3,2482.0,5.4,631538,543272,547379,467793,514917,608070,596019,640458,592261,656669,55.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76,2,Slider,8,6,6,8,6,8,6,8,Infield shift,Standard


In [9]:
df.shape

(263584, 90)

### Find features to use for model

#### Pitchers

In [5]:
pitcheres = df.player_name.value_counts().to_frame().reset_index().rename(columns={'index': 'pitcher', 'player_name': 'count'})
pitcheres.head()

,pitcher,count
0,Lance Lynn,1408
1,Shane Bieber,1238
2,German Marquez,1226
3,Lucas Giolito,1214
4,Max Scherzer,1213


In [6]:
pitcheres['count'].mean()

359.5961800818554

In [7]:
pitcheres['count'].min()

2

In [8]:
pitcheres['count'].describe()

count     733.000000
mean      359.596180
std       299.813281
min         2.000000
25%       118.000000
50%       302.000000
75%       470.000000
max      1408.000000
Name: count, dtype: float64

In [10]:
df.player_name.value_counts().to_frame().mean()

player_name    359.59618
dtype: float64

#### Release speed

In [11]:
df.release_speed.value_counts().head(10)

93.5    2154
93.9    2142
93.1    2142
93.3    2128
93.4    2123
93.8    2120
93.2    2113
94.1    2109
92.7    2100
92.9    2096
Name: release_speed, dtype: int64

In [12]:
rl_speed = df.release_speed.value_counts().to_frame().reset_index().rename(columns={'index': 'speed', 'release_speed': 'count'})

In [13]:
rl_speed.speed.describe()

count    466.000000
mean      77.944635
std       14.872838
min       38.100000
25%       67.125000
50%       78.750000
75%       90.375000
max      102.200000
Name: speed, dtype: float64

In [14]:
rl_speed.loc[rl_speed.speed.isnull()]

,speed,count


#### Pitch outcome

In [15]:
df.type.value_counts()

S    121964
B     97648
X     43972
Name: type, dtype: int64

#### Outcome per at bat

In [16]:
df.events.value_counts()

field_out                     25530
strikeout                     15543
single                         9071
walk                           5890
double                         2823
home_run                       2304
force_out                      1308
grounded_into_double_play      1235
hit_by_pitch                    821
field_error                     513
sac_fly                         399
triple                          241
double_play                     150
fielders_choice                 149
sac_bunt                        126
fielders_choice_out             118
caught_stealing_2b               71
strikeout_double_play            43
interf_def                       35
pickoff_caught_stealing_2b       14
pickoff_1b                       12
other_out                        11
pickoff_2b                        7
caught_stealing_3b                6
caught_stealing_home              4
sac_fly_double_play               3
run                               2
triple_play                 

#### Description per pitch outcome

In [19]:
df.description.value_counts()

ball                       89937
foul                       44716
called_strike              44320
hit_into_play              28264
swinging_strike            28026
hit_into_play_no_out        9685
blocked_ball                6874
hit_into_play_score         6029
foul_tip                    2427
swinging_strike_blocked     1968
hit_by_pitch                 821
foul_bunt                    434
missed_bunt                   62
pitchout                      16
bunt_foul_tip                  5
Name: description, dtype: int64

###### Pitch zones

In [23]:
df.zone

0          1.0
1         13.0
2         13.0
3         13.0
4         13.0
          ... 
263579     5.0
263580     7.0
263581     9.0
263582    11.0
263583    11.0
Name: zone, Length: 263584, dtype: float64

In [24]:
df[df.zone.isnull()][['description']]

,description
20677,ball
20682,ball
70022,hit_into_play
70023,blocked_ball
70024,called_strike
...,...
233601,foul
233738,foul
234935,ball
234937,ball


In [124]:
df.pitch_type.value_counts()

FF    89618
SL    46849
SI    42768
CH    31131
CU    24679
FC    17404
KC     5939
FS     4528
CS      191
KN       23
FO        3
Name: pitch_type, dtype: int64

##### Pitch legend

- AB Automatic Ball
- AS Automatic Strike
- CH Change-​up
- CU Curveball
- EP Eephus
- FC Cutter
- FF Four-Seam Fastball
- FO Forkball
- FS Splitter
- FT Two-Seam Fastball (synonymous with SI)
- GY Gyroball
- IN Intentional Ball
- KC Knuckle Curve
- KN Knuckleball
- NP No Pitch
- PO Pitchout
- SC Screwball
- SI Sinker (synonymous with FT)
- SL Slider
- UN Unknown

In [117]:
df.groupby(['pitch_type', 'pitch_name']).size().to_frame().reset_index().rename(columns={0: 'count'}).sort_values(by='count', ascending=False)

,pitch_type,pitch_name,count
4,FF,4-Seam Fastball,89618
10,SL,Slider,46849
9,SI,Sinker,42768
0,CH,Changeup,31131
2,CU,Curveball,24679
3,FC,Cutter,17404
7,KC,Knuckle Curve,5939
6,FS,Split-Finger,4528
1,CS,Curveball,191
8,KN,Knuckleball,23


In [26]:
x = df[~df.zone.isnull()].copy()

##### Hot encode non numeric values for y values

In [29]:
def hot_encode_des(x):
    if x == 'hit_into_play_no_out' or x == 'hit_into_play_score':
        return 1
    return 0

In [30]:
def hot_encode_event(x):
    if x == 'single' or x == 'double' or x == 'triple' or x == 'home_run':
        return 1
    return 0

In [31]:
x['norm_zone'] = x['zone'].astype(str)

### Selected features

In [32]:
x_norm = x[['player_name', 'pitch_type', 'release_speed', 'norm_zone']]

In [34]:
y = x['description'].apply(lambda x: hot_encode_des(x))

##### Hot encode non numeric data

In [35]:
x = pd.get_dummies(x_norm)

##### Look at y values

In [37]:
hits = [i for i in y if i == 1]
print(len(hits))
print(len(y))
print((len(hits)/len(y) )* 100) 

15706
263393
5.962952698059554


##### Split data

In [38]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30)

##### view train data

In [89]:
X_train.head()

,release_speed,player_name_A.J. Cole,player_name_A.J. Minter,player_name_AJ Ramos,player_name_Aaron Barrett,player_name_Aaron Bummer,player_name_Aaron Civale,player_name_Aaron Fletcher,player_name_Aaron Loup,player_name_Aaron Nola,player_name_Aaron Slegers,player_name_Adam Cimber,player_name_Adam Kolarek,player_name_Adam Morgan,player_name_Adam Ottavino,player_name_Adam Plutko,player_name_Adam Wainwright,player_name_Adbert Alzolay,player_name_Adonis Medina,player_name_Adrian Houser,player_name_Adrian Morejon,player_name_Albert Abreu,player_name_Alec Mills,player_name_Alex Claudio,player_name_Alex Cobb,player_name_Alex Colome,player_name_Alex McRae,player_name_Alex Reyes,player_name_Alex Vesia,player_name_Alex Wood,player_name_Alex Young,player_name_Amir Garrett,player_name_Andre Scrubb,player_name_Andrew Chafin,player_name_Andrew Heaney,player_name_Andrew Kittredge,player_name_Andrew Miller,player_name_Andrew Suarez,player_name_Andrew Triggs,player_name_Angel Perdomo,player_name_Anibal Sanchez,player_name_Anthony Banda,player_name_Anthony Bass,player_name_Anthony Bemboom,player_name_Anthony Castro,player_name_Anthony DeSclafani,player_name_Anthony Kay,player_name_Anthony Misiewicz,player_name_Antonio Santos,player_name_Antonio Senzatela,player_name_Archie Bradley,player_name_Ariel Jurado,player_name_Aroldis Chapman,player_name_Artie Lewicki,player_name_Asher Wojciechowski,player_name_Ashton Goudeau,player_name_Austin Adams,player_name_Austin Brice,player_name_Austin Davis,player_name_Austin Gomber,player_name_Austin Voth,player_name_Beau Burrows,player_name_Ben Braymer,player_name_Ben Heller,player_name_Bernardo Flores Jr.,player_name_Blake Cederlind,player_name_Blake Parker,player_name_Blake Snell,player_name_Blake Taylor,player_name_Blake Treinen,player_name_Bobby Wahl,player_name_Brad Boxberger,player_name_Brad Brach,player_name_Brad Hand,player_name_Brad Keller,player_name_Brad Peacock,player_name_Brad Wieck,player_name_Brady Lail,player_name_Brady Singer,player_name_Brailyn Marquez,player_name_Branden Kline,player_name_Brandon Bailey,player_name_Brandon Bielak,player_name_Brandon Brennan,player_name_Brandon Kintzler,player_name_Brandon Leibrandt,player_name_Brandon Waddell,player_name_Brandon Woodruff,player_name_Brandon Workman,player_name_Braxton Garrett,player_name_Brent Suter,player_name_Brett Anderson,player_name_Brett Eibner,player_name_Brett Martin,player_name_Brian Moran,player_name_Brock Holt,player_name_Brooks Kriske,player_name_Brooks Raley,player_name_Bruce Zimmermann,player_name_Brusdar Graterol,player_name_Bryan Abreu,player_name_Bryan Garcia,player_name_Bryan Holaday,player_name_Bryan Shaw,player_name_Bryse Wilson,player_name_Buck Farmer,player_name_Burch Smith,player_name_Cal Quantrill,player_name_Caleb Baragar,player_name_Caleb Ferguson,player_name_Caleb Smith,player_name_Caleb Thielbar,player_name_Cam Bedrosian,player_name_Cam Hill,player_name_Carl Edwards Jr.,player_name_Carlos Carrasco,player_name_Carlos Estevez,player_name_Carlos Hernandez,player_name_Carlos Martinez,player_name_Carlos Rodon,player_name_Carlos Sanabria,player_name_Carson Fulmer,player_name_Carson Kelly,player_name_Casey Mize,player_name_Casey Sadler,player_name_Cesar Valdez,player_name_Chad Green,player_name_Chad Kuhl,player_name_Chad Sobotka,player_name_Chance Adams,player_name_Charlie Culberson,player_name_Charlie Morton,player_name_Chase Anderson,player_name_Chase De Jong,player_name_Chasen Shreve,player_name_Chaz Roe,player_name_Chi Chi Gonzalez,player_name_Chris Bassitt,player_name_Chris Devenski,player_name_Chris Martin,player_name_Chris Mazza,player_name_Chris Paddack,player_name_Chris Rusin,player_name_Chris Stratton,player_name_Cionel Perez,player_name_Clarke Schmidt,player_name_Clay Holmes,player_name_Clayton Kershaw,player_name_Codi Heuer,player_name_Cody Carroll,player_name_Cody Ponce,player_name_Cody Reed,player_name_Cody Stashak,player_name_Cole Hamels,player_name_Cole Irvin,player_name_Cole Sulser,player_name_Colin Rea,

##### View shape

In [90]:
X_train.shape

(184375, 758)

In [91]:
X_test.head()

,release_speed,player_name_A.J. Cole,player_name_A.J. Minter,player_name_AJ Ramos,player_name_Aaron Barrett,player_name_Aaron Bummer,player_name_Aaron Civale,player_name_Aaron Fletcher,player_name_Aaron Loup,player_name_Aaron Nola,player_name_Aaron Slegers,player_name_Adam Cimber,player_name_Adam Kolarek,player_name_Adam Morgan,player_name_Adam Ottavino,player_name_Adam Plutko,player_name_Adam Wainwright,player_name_Adbert Alzolay,player_name_Adonis Medina,player_name_Adrian Houser,player_name_Adrian Morejon,player_name_Albert Abreu,player_name_Alec Mills,player_name_Alex Claudio,player_name_Alex Cobb,player_name_Alex Colome,player_name_Alex McRae,player_name_Alex Reyes,player_name_Alex Vesia,player_name_Alex Wood,player_name_Alex Young,player_name_Amir Garrett,player_name_Andre Scrubb,player_name_Andrew Chafin,player_name_Andrew Heaney,player_name_Andrew Kittredge,player_name_Andrew Miller,player_name_Andrew Suarez,player_name_Andrew Triggs,player_name_Angel Perdomo,player_name_Anibal Sanchez,player_name_Anthony Banda,player_name_Anthony Bass,player_name_Anthony Bemboom,player_name_Anthony Castro,player_name_Anthony DeSclafani,player_name_Anthony Kay,player_name_Anthony Misiewicz,player_name_Antonio Santos,player_name_Antonio Senzatela,player_name_Archie Bradley,player_name_Ariel Jurado,player_name_Aroldis Chapman,player_name_Artie Lewicki,player_name_Asher Wojciechowski,player_name_Ashton Goudeau,player_name_Austin Adams,player_name_Austin Brice,player_name_Austin Davis,player_name_Austin Gomber,player_name_Austin Voth,player_name_Beau Burrows,player_name_Ben Braymer,player_name_Ben Heller,player_name_Bernardo Flores Jr.,player_name_Blake Cederlind,player_name_Blake Parker,player_name_Blake Snell,player_name_Blake Taylor,player_name_Blake Treinen,player_name_Bobby Wahl,player_name_Brad Boxberger,player_name_Brad Brach,player_name_Brad Hand,player_name_Brad Keller,player_name_Brad Peacock,player_name_Brad Wieck,player_name_Brady Lail,player_name_Brady Singer,player_name_Brailyn Marquez,player_name_Branden Kline,player_name_Brandon Bailey,player_name_Brandon Bielak,player_name_Brandon Brennan,player_name_Brandon Kintzler,player_name_Brandon Leibrandt,player_name_Brandon Waddell,player_name_Brandon Woodruff,player_name_Brandon Workman,player_name_Braxton Garrett,player_name_Brent Suter,player_name_Brett Anderson,player_name_Brett Eibner,player_name_Brett Martin,player_name_Brian Moran,player_name_Brock Holt,player_name_Brooks Kriske,player_name_Brooks Raley,player_name_Bruce Zimmermann,player_name_Brusdar Graterol,player_name_Bryan Abreu,player_name_Bryan Garcia,player_name_Bryan Holaday,player_name_Bryan Shaw,player_name_Bryse Wilson,player_name_Buck Farmer,player_name_Burch Smith,player_name_Cal Quantrill,player_name_Caleb Baragar,player_name_Caleb Ferguson,player_name_Caleb Smith,player_name_Caleb Thielbar,player_name_Cam Bedrosian,player_name_Cam Hill,player_name_Carl Edwards Jr.,player_name_Carlos Carrasco,player_name_Carlos Estevez,player_name_Carlos Hernandez,player_name_Carlos Martinez,player_name_Carlos Rodon,player_name_Carlos Sanabria,player_name_Carson Fulmer,player_name_Carson Kelly,player_name_Casey Mize,player_name_Casey Sadler,player_name_Cesar Valdez,player_name_Chad Green,player_name_Chad Kuhl,player_name_Chad Sobotka,player_name_Chance Adams,player_name_Charlie Culberson,player_name_Charlie Morton,player_name_Chase Anderson,player_name_Chase De Jong,player_name_Chasen Shreve,player_name_Chaz Roe,player_name_Chi Chi Gonzalez,player_name_Chris Bassitt,player_name_Chris Devenski,player_name_Chris Martin,player_name_Chris Mazza,player_name_Chris Paddack,player_name_Chris Rusin,player_name_Chris Stratton,player_name_Cionel Perez,player_name_Clarke Schmidt,player_name_Clay Holmes,player_name_Clayton Kershaw,player_name_Codi Heuer,player_name_Cody Carroll,player_name_Cody Ponce,player_name_Cody Reed,player_name_Cody Stashak,player_name_Cole Hamels,player_name_Cole Irvin,player_name_Cole Sulser,player_name_Colin Rea,

#### Build probability model

In [42]:
clf = LogisticRegression(max_iter=1000, solver='saga', penalty='elasticnet', l1_ratio=0.5)

In [43]:
clf.fit(X_train, y_train)

/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=0.5, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='elasticnet',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [45]:
pred = clf.predict(X_test)

#### As expected, the classification is predicting all zeros (no hit).
- We don't really care about the classification since we want to find the probilities
- We'll use the classification to score the model

In [92]:
pred

array([0, 0, 0, ..., 0, 0, 0])

In [93]:
for i in pred:
    if i == 1:
        print("hit")

##### Check accuracy score of model 

In [94]:
score = metrics.accuracy_score(y_test, pred)

In [95]:
score

0.939697284163102

###### describe the relationship between a predictor variable and the response. 

In [50]:
clf.coef_[0]

array([-5.71470330e-03,  0.00000000e+00, -3.20906179e-01,  0.00000000e+00,
        0.00000000e+00, -2.59685048e-01,  1.06284799e-01,  8.09563004e-02,
       -4.56274415e-01, -1.22772188e-01, -2.44934317e-01,  3.27345566e-01,
       -1.01521490e-01, -1.41986047e-01,  0.00000000e+00,  1.18702100e-01,
       -4.32358007e-02, -3.17612355e-01, -3.25457023e-03,  1.54753634e-01,
       -1.34535229e-01,  0.00000000e+00, -7.04335677e-02, -1.28673468e-01,
        0.00000000e+00, -4.21989949e-01, -1.31433944e-02, -3.21358553e-02,
        0.00000000e+00,  0.00000000e+00,  2.68399040e-01,  0.00000000e+00,
       -2.49022182e-01,  4.71898170e-02,  1.00200577e-02, -8.63911569e-02,
        0.00000000e+00, -2.13082364e-02,  0.00000000e+00,  0.00000000e+00,
        2.52073230e-01,  2.53849782e-01, -2.89956387e-01,  0.00000000e+00,
        0.00000000e+00,  4.00236301e-01, -8.83988453e-02,  1.01919330e-01,
        4.55152259e-01,  2.32411264e-01,  2.55777200e-01,  7.05228015e-01,
       -3.37946465e-01,  

###### Find the probs 

In [100]:
probs = clf.predict_proba(X=X_test)

In [101]:
probs = probs * 100

In [102]:
hit_prob = pd.DataFrame(probs, columns=['no-hit', 'hit'])

In [103]:
hit_prob.head()

,no-hit,hit
0,91.756307,8.243693
1,98.056853,1.943147
2,85.191628,14.808372
3,90.842875,9.157125
4,89.570632,10.429368


In [63]:
hit_prob.shape

(79018, 2)

###### Build out dataframe with probs and outcomes

In [64]:
x_df = X_test.copy()

In [65]:
y_df = pd.DataFrame(y_test)

In [66]:
y_df

,description
96462,0
55382,0
76838,0
156632,0
20428,1
...,...
179000,0
142333,0
225582,0
215283,0


In [67]:
x_df['description'] = y_df

In [105]:
x_df.head()[['release_speed', 'description' ]]

,release_speed,description
0,88.7,0
1,91.7,0
2,86.0,0
3,92.3,0
4,96.8,1


In [69]:
x_df.reset_index(inplace=True)

In [70]:
final_df = x_df.join(hit_prob)

#### Find the higest probability of a hit

In [75]:
final_df.hit.max()

30.944500842142393

In [106]:
final_df[final_df.hit == final_df.hit.max()][['index', 'release_speed', 'description', 'no-hit', 'hit']]

,index,release_speed,description,no-hit,hit
43566,78983,89.1,1,69.055499,30.944501


In [122]:
df.iloc[[78983]][['pitch_type', 'pitch_name','release_speed', 'zone', 'events', 'description']]

,pitch_type,pitch_name,release_speed,zone,events,description
78983,SI,Sinker,89.1,5.0,single,hit_into_play_no_out


#### find prob of all X

In [167]:
all_x = df.copy()

In [168]:
all_x['norm_zone'] = all_x['zone'].astype(str)

In [169]:
features = all_x[['player_name', 'pitch_type', 'release_speed', 'norm_zone']]

In [170]:
features[features.player_name.isnull() == True]

,player_name,pitch_type,release_speed,norm_zone


In [172]:
features[features.release_speed.isnull() == True]['player_name'].value_counts()

Gerrit Cole          114
Thomas Eshelman       41
Alex Cobb             28
Brock Holt             2
Taijuan Walker         2
Kris Bubic             1
Evan Marshall          1
Yoan Lopez             1
Richard Rodriguez      1
Name: player_name, dtype: int64

In [173]:
features[features.pitch_type.isnull() == True]

,player_name,pitch_type,release_speed,norm_zone
20677,Brock Holt,NaN,NaN,nan
20682,Brock Holt,NaN,NaN,nan
70022,Gerrit Cole,NaN,NaN,nan
70023,Gerrit Cole,NaN,NaN,nan
70024,Gerrit Cole,NaN,NaN,nan
...,...,...,...,...
233601,Evan Marshall,NaN,NaN,nan
233738,Kris Bubic,NaN,NaN,nan
234935,Taijuan Walker,NaN,NaN,nan
234937,Taijuan Walker,NaN,NaN,nan


In [184]:
#####

In [175]:
features.dropna(inplace=True)

<ipython-input-175-aa60b42c2688>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.dropna(inplace=True)


In [176]:
encoded_x = pd.get_dummies(features)

In [177]:
probs = clf.predict_proba(encoded_x)

In [178]:
probs = probs*100

In [179]:
hit_prob = pd.DataFrame(probs, columns=['no-hit', 'hit'])

In [180]:
hit_prob

,no-hit,hit
0,96.820104,3.179896
1,98.782501,1.217499
2,98.785248,1.214752
3,98.783189,1.216811
4,98.789355,1.210645
...,...,...
263128,86.605824,13.394176
263129,91.433504,8.566496
263130,92.010817,7.989183
263131,98.292477,1.707523


In [165]:
hit_prob.max()

no-hit    99.386774
hit       30.968930
dtype: float64

In [ ]:
##### 

In [185]:
encoded_x.head()

,release_speed,player_name_A.J. Cole,player_name_A.J. Minter,player_name_AJ Ramos,player_name_Aaron Barrett,player_name_Aaron Bummer,player_name_Aaron Civale,player_name_Aaron Fletcher,player_name_Aaron Loup,player_name_Aaron Nola,player_name_Aaron Slegers,player_name_Adam Cimber,player_name_Adam Kolarek,player_name_Adam Morgan,player_name_Adam Ottavino,player_name_Adam Plutko,player_name_Adam Wainwright,player_name_Adbert Alzolay,player_name_Adonis Medina,player_name_Adrian Houser,player_name_Adrian Morejon,player_name_Albert Abreu,player_name_Alec Mills,player_name_Alex Claudio,player_name_Alex Cobb,player_name_Alex Colome,player_name_Alex McRae,player_name_Alex Reyes,player_name_Alex Vesia,player_name_Alex Wood,player_name_Alex Young,player_name_Amir Garrett,player_name_Andre Scrubb,player_name_Andrew Chafin,player_name_Andrew Heaney,player_name_Andrew Kittredge,player_name_Andrew Miller,player_name_Andrew Suarez,player_name_Andrew Triggs,player_name_Angel Perdomo,player_name_Anibal Sanchez,player_name_Anthony Banda,player_name_Anthony Bass,player_name_Anthony Bemboom,player_name_Anthony Castro,player_name_Anthony DeSclafani,player_name_Anthony Kay,player_name_Anthony Misiewicz,player_name_Antonio Santos,player_name_Antonio Senzatela,player_name_Archie Bradley,player_name_Ariel Jurado,player_name_Aroldis Chapman,player_name_Artie Lewicki,player_name_Asher Wojciechowski,player_name_Ashton Goudeau,player_name_Austin Adams,player_name_Austin Brice,player_name_Austin Davis,player_name_Austin Gomber,player_name_Austin Voth,player_name_Beau Burrows,player_name_Ben Braymer,player_name_Ben Heller,player_name_Bernardo Flores Jr.,player_name_Blake Cederlind,player_name_Blake Parker,player_name_Blake Snell,player_name_Blake Taylor,player_name_Blake Treinen,player_name_Bobby Wahl,player_name_Brad Boxberger,player_name_Brad Brach,player_name_Brad Hand,player_name_Brad Keller,player_name_Brad Peacock,player_name_Brad Wieck,player_name_Brady Lail,player_name_Brady Singer,player_name_Brailyn Marquez,player_name_Branden Kline,player_name_Brandon Bailey,player_name_Brandon Bielak,player_name_Brandon Brennan,player_name_Brandon Kintzler,player_name_Brandon Leibrandt,player_name_Brandon Waddell,player_name_Brandon Woodruff,player_name_Brandon Workman,player_name_Braxton Garrett,player_name_Brent Suter,player_name_Brett Anderson,player_name_Brett Eibner,player_name_Brett Martin,player_name_Brian Moran,player_name_Brock Holt,player_name_Brooks Kriske,player_name_Brooks Raley,player_name_Bruce Zimmermann,player_name_Brusdar Graterol,player_name_Bryan Abreu,player_name_Bryan Garcia,player_name_Bryan Holaday,player_name_Bryan Shaw,player_name_Bryse Wilson,player_name_Buck Farmer,player_name_Burch Smith,player_name_Cal Quantrill,player_name_Caleb Baragar,player_name_Caleb Ferguson,player_name_Caleb Smith,player_name_Caleb Thielbar,player_name_Cam Bedrosian,player_name_Cam Hill,player_name_Carl Edwards Jr.,player_name_Carlos Carrasco,player_name_Carlos Estevez,player_name_Carlos Hernandez,player_name_Carlos Martinez,player_name_Carlos Rodon,player_name_Carlos Sanabria,player_name_Carson Fulmer,player_name_Carson Kelly,player_name_Casey Mize,player_name_Casey Sadler,player_name_Cesar Valdez,player_name_Chad Green,player_name_Chad Kuhl,player_name_Chad Sobotka,player_name_Chance Adams,player_name_Charlie Culberson,player_name_Charlie Morton,player_name_Chase Anderson,player_name_Chase De Jong,player_name_Chasen Shreve,player_name_Chaz Roe,player_name_Chi Chi Gonzalez,player_name_Chris Bassitt,player_name_Chris Devenski,player_name_Chris Martin,player_name_Chris Mazza,player_name_Chris Paddack,player_name_Chris Rusin,player_name_Chris Stratton,player_name_Cionel Perez,player_name_Clarke Schmidt,player_name_Clay Holmes,player_name_Clayton Kershaw,player_name_Codi Heuer,player_name_Cody Carroll,player_name_Cody Ponce,player_name_Cody Reed,player_name_Cody Stashak,player_name_Cole Hamels,player_name_Cole Irvin,player_name_Cole Sulser,player_name_Colin Rea,

In [186]:
encoded_x.shape

(263133, 758)

In [187]:
output_x = encoded_x.copy()
output_x.iloc[0:0] 

In [ ]:
# save the model to disk
filename = 'pitch_prob.sav'
pickle.dump(clf, open(filename, 'wb'))